In [ ]:
import numpy as np
import pandas as pd
import random  # utilisé pour le choix aléatoire d'une ligne de X pour le gradient_stochastique

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cd /content/drive/MyDrive/

* Veuillez mettre en œuvre les tâches suivantes :

1. Un fichier csv contenant l'ensemble de données sera partagé, téléchargez-le et téléchargez-le.

2. Lisez le fichier csv en utilisant Pandas.

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/apprentissage_supervise/classwork-data.csv") # write the path of the data file here


In [ ]:
data.head()

3. Créez une variable X en numpy array pour les colonnes d'entrée (col1, col2, col3, col4).

Transformer le tableau pandas en Numpy

In [ ]:
X =  data.iloc[:,1:5].values 
X

In [ ]:
X.shape

4. Créez une variable Y pour la colonne de sortie (cible).

In [ ]:
Y  =  data.iloc[:,-1].values

In [ ]:
Y

In [ ]:
Y.shape

In [ ]:
Y = Y.reshape(Y.shape[0],1)

In [ ]:
Y.shape

In [ ]:
Y

In [ ]:
#################################################################################################################

5. Divisez l'ensemble de données en deux ensembles, l'ensemble de formation et l'ensemble de test.
* Vérifiez en ligne [ici] (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) comment utiliser la fonction **train_test_split** de la bibliothèque sklearn.

In [ ]:
# You need to find:   X_train, X_test, Y_train, Y_test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
Y_train.shape

In [ ]:
#################################################################################################################

6. Utilisez l'erreur quadratique moyenne pour rendre compte de la performance du modèle de régression linéaire.

$$
MSE = \frac{\sum{(target - prediction)^2}}{n}
$$

In [ ]:
# define the MSE fun
def MSE(y_true, y_pred):
  # calculate the MSE and return it
  n = len(y_true)
  mse = (np.sum(np.square(y_true - y_pred)))*(1/n)
  return mse

In [ ]:
#################################################################################################################

7. Implémenter la fonction MLE, qui calcule l'estimateur du maximum de vraisemblance pour les données données.

Calculez le paramètre du modèle en utilisant la solution en forme fermée comme indiqué dans l'équation suivante :

$$
\boldsymbol w = (\boldsymbol X^T\boldsymbol X)^{-1}\boldsymbol X^T\boldsymbol Y
$$

X représente les entrées, et Y les cibles.

In [ ]:
def NormalEquation(inputs, targets):
  # calculate the parameter w and return it
  w = np.linalg.inv(inputs.T.dot(inputs)).dot((inputs.T).dot(targets))
  return w

In [ ]:
#################################################################################################################

Évaluons le modèle en prenant un exemple des données de test, puis calculons l'EQM entre la prédiction du modèle et la cible.

In [ ]:
w = NormalEquation(X_train, Y_train)      # All the parameter estimation should be done using  only the training data.

In [ ]:
w

In [ ]:
def predict(X_test, w):
  y_pred = X_test.dot(w)
  return y_pred

In [ ]:
Y_pred = predict(X_test, w)

In [ ]:
Y_pred

In [ ]:
print(MSE(Y_test, Y_pred))

#### Methode descende de gradient

In [ ]:
def gradient(X, Y, o, learning_rate) :
  n = len(Y) # la taille de Y
  return learning_rate*((X.T).dot(X.dot(o)  - Y))*(1/n)

In [ ]:
def fit(X_train,Y_train, lr) :
  w = np.zeros((X_train.shape[1], 1))  
  max_iters = 20 # max number of iterations

  for  i in range(0,max_iters): 
      w = w - gradient(X_train, Y_train, w, lr)
      print(" iteration ", i ," : current_w = \n", w)
  print("  Local Minimum w :  \n", w)
  return w

In [ ]:
o = fit(X_train,Y_train, 0.9)

In [ ]:
y_gradient = predict(X_test, o)

In [ ]:
y_gradient

In [ ]:
mse = MSE(Y_test, y_gradient)
print(MSE)

### Creer une classe et inserer les differentes methodes mse, predict, train implementer plus haut.

In [ ]:
class LR:
   
  def __init__(self, lr):
    self.lr = lr

  def fit2(X_train, Y_train, n, lr): # descente gradient stochastique
      current_o = np.zeros((X_train.shape[1] , 1))
      max_iters = 15 # max number of iterations

      for i in range(0,max_iters) :
          previous_o = current_o 
          rd_nmber = random.choice(np.arange(0, X_train.shape[0]))
          X_line = X_train[rd_nmber, :].reshape([1 , X_train.shape[1] ]) 
          Y_line = Y_train[rd_nmber, 0:1]
          current_o = previous_o - lr*((X_line.T).dot((X_line).dot(previous_o)  - Y_line))*(1/n)
      return current_o
    
  def fit1(X_train,Y_train, n, lr) : # descente gradient batch
      current_o = np.zeros((X_train.shape[1], 1)) 
      max_iters = 15 # max number of iterations
      for  i in range(0, max_iters):
          previous_o = current_o 
          current_o = previous_o - lr*((X_train.T).dot(X_train.dot(previous_o) - Y_train))*(1/n)
      return current_o

  def fit(self, X_train, Y_train): # cette methode renvoie w_batch et w_stochastique
    n = len(Y_train) # taille du jeu de donnee 
    self.w_batch = LR.fit1(X_train,Y_train, n , self.lr) # descente gradient batch
    self.w_stch = LR.fit2(X_train,Y_train,  n , self.lr) # descente gradient stochastique
    return True

  def predict(self, X_test):
    return (X_test.dot(self.w_batch),  (X_test).dot(self.w_stch))

  def mse(self, y_true, y_pred): 
    n = len(y_true)
    mse = (np.sum(np.square(y_true - y_pred)))*(1/n)
    print("MSE : ", mse ) 


In [ ]:
model = LR(lr= 0.000001)
print(model.lr)
model.fit(X_train= X_train, Y_train= Y_train)
y_pre_btch, y_pre_stch  = model.predict(X_test = X_test)
print(y_pre_btch) # valeur de Y selon la methode du gradient batch
print(y_pre_stch) #valeur de Y selon la methode du gradient stochastique
model.mse(y_true= Y_test, y_pred= y_pre_btch) # mse batch
model.mse(y_true= Y_test, y_pred= y_pre_stch) # mse stochastique